In [ ]:
from corda import CORDA
import pandas as pd
import numpy as np
from cobra.io import read_sbml_model, write_sbml_model
from cobra.flux_analysis import fastcc
import os


In [ ]:
# Load Recon3D
model = read_sbml_model('/home/jiang817/cybergut_project/General_models/Recon3D.xml')

In [ ]:
# Set exchagne reaction boundries as (-100, 100)
for i in range (len(model.reactions)):
    if model.reactions[i].id.startswith('EX_'):
        model.reactions[i].bounds = (-100, 100)

In [ ]:
rxn_score = pd.read_csv('/home/jiang817/cybergut_project/Draft_model/Recon3D_input/Recon3D_BIGG_iMAT_INPUT/rxn_score_corda.csv')

In [ ]:
# Set reaction score according to original values
for idx in range(len(rxn_score)):
    if rxn_score.iloc[idx, 1] > 0:
        print(rxn_score.iloc[idx, 1])
        rxn_score.iloc[idx, 1] = 3
    if rxn_score.iloc[idx, 1] == 0:
        print(rxn_score.iloc[idx, 1])
        rxn_score.iloc[idx, 1] = -1

rxn_score.fillna(0, inplace=True)

confidence = {}
for i in range(len(rxn_score)):
    key = rxn_score.iloc[i, 0]
    value = rxn_score.iloc[i, 1]
    print(key, value)
    confidence[key] = value

In [ ]:
# Load metabolites of BIOMASS_maaintenance 
Met = pd.read_csv('//home/jiang817/cybergut_project/Draft_model/Recon3D_input/Recon3D_BIGG_CORDA_INPUT/KeyMets_Recon3D_XML.txt', header=0)
Met = Met['Met'].to_list()

In [ ]:
# Run CORDA
ts_model = CORDA(model, confidence, Met)
ts_model.build()

In [ ]:
# Obtain 
for i in ts_model.included:
    if ts_model.included[i] == False:
        model.remove_reactions(i, remove_orphans=True)
        

In [ ]:
# save draft reconstuction
write_sbml_model(model, '/home/jiang817/cybergut_project/Draft_model/Draft_model_with_CORDA/draft_model_CORDA_BIGG.xml')

In [ ]:
# run fastcc to obtain consistent reconstruction 
model_fastcc = fastcc(model)

# save consistent reconstuction
write_sbml_model(model_fastcc, '/home/jiang817/cybergut_project/Gapfilling/fastcc_outputs/reconstruction_CORDA_fastcc.xml')